# Evaluation of candidate schemes

This notebook contains the calculations to evaluate the performance of candidate post-quantum
signature schemes in the RPKI. It is the underlying data for the tables in sections 4.4 and 4.5.
The methodology and constants are based on section 4.2, and benchmarks of post-quantum schemes
come from the overview of [PQshield's signatures zoo](https://github.com/pqshield/nist-sigs-zoo).

In [1]:
import pandas as pd

# Load data from parametersets.csv from https://github.com/PQShield/nist-sigs-zoo/tree/d525707d3010edfd5d2f2b32e95dc97388057c77/data/parametersets.csv
# This is a fork containing corrected values for FAEST-EM-128s.
df = pd.read_csv("parametersets.csv")

# Drop schemes for which no benchmark in cycles exists.
df = df.dropna(subset=["signing (cycles)", "verification (cycles)"])

In [2]:
# Define some constants based on measurements from the thesis.

CPU_FREQ = 2_500_000_000  # 2.5 GHz, assumption for converting cycles to seconds.
SNAPSHOT_N_PK = 393427  # Number of public keys, from snapshot in table 4.1.
SNAPSHOT_N_SIG = 788916  # Number of signatures, from snapshot in table 4.1.
SNAPSHOT_SIZE = 838030450  # Size of the snapshot in bytes, from table 4.1.
VERIFICATION_CPU_TIME = 13  # CPU time (s) for verification, from 4.2.2.1.
DOWNLOAD_TIME_HIGH = 240  # Download time (s) from 4.2.1.2.
DOWNLOAD_TIME_LOW = 14.5  # Download time (s) from 4.2.1.3.

In [3]:
# Extract the RSA-2048 scheme as reference for the calculations.
rsa2048 = df[(df["Scheme"] == "RSA") & (df["Parameterset"] == "2048")].to_dict("records")[0]
ed25519 = df[(df["Scheme"] == "EdDSA") & (df["Parameterset"] == "Ed25519")].to_dict("records")[0]
rsa3072 = rsa2048.copy()
rsa3072["verification (cycles)"] = 2.2 * rsa2048["verification (cycles)"]
rsa3072["signing (cycles)"] = None
rsa3072["pk size"] = 400
rsa3072["sig size"] = 384
rsa3072["Parameterset"] = "3072"

# Define traditional components for hybrid schemes
traditional_components = {
    "RSA-2048": rsa2048,
    "RSA-3072": rsa3072,
    "Ed25519": ed25519,
}

# Create a list to store new rows
hybrid_rows = []

# Iterate through each row in the dataframe
for _, row in df.iterrows():
    for component_name, component in traditional_components.items():
        hybrid_row = row.copy()
        hybrid_row["Scheme"] = f"{row['Scheme']}-{row['Parameterset']}"
        hybrid_row["Parameterset"] = component_name
        hybrid_row["pk size"] += component["pk size"]
        hybrid_row["sig size"] += component["sig size"]
        hybrid_row["signing (cycles)"] = None
        hybrid_row["verification (cycles)"] += component["verification (cycles)"]
        hybrid_rows.append(hybrid_row)

# Append the hybrid rows to the original dataframe
df = pd.concat([df, pd.DataFrame(hybrid_rows)], ignore_index=True)

/tmp/ipykernel_14574/3839629625.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame(hybrid_rows)], ignore_index=True)


In [4]:
# Add columns related to use in the RPKI.
df["Total size (B)"] = SNAPSHOT_SIZE \
    + SNAPSHOT_N_PK * df["pk size"] - SNAPSHOT_N_PK * rsa2048["pk size"] \
    + SNAPSHOT_N_SIG * df["sig size"] - SNAPSHOT_N_SIG * rsa2048["sig size"]

df["Total size (MB)"] = df["Total size (B)"] / 1_000_000
df["Size factor"] = df["Total size (B)"] / SNAPSHOT_SIZE
df["Download time (high) (s)"] = DOWNLOAD_TIME_HIGH * df["Size factor"]
df["Download time (low) (s)"] = DOWNLOAD_TIME_LOW * df["Size factor"]
df["Verification time factor"] = df["verification (cycles)"] / rsa2048["verification (cycles)"]
df["Verification time (s)"] = VERIFICATION_CPU_TIME * df["Verification time factor"] 
df["Signing time (s)"] = df["signing (cycles)"] / CPU_FREQ

print(df)

             Scheme      Parameterset Security level  pk size  sig size  \
0             CROSS     R-SDP 5 small              5      153     50818   
1             CROSS  R-SDP 5 balanced              5      153     53527   
2             CROSS     R-SDP 3 small              3      115     28391   
3             CROSS  R-SDP 3 balanced              3      115     29853   
4             CROSS      R-SDP 5 fast              5      153     74590   
..              ...               ...            ...      ...       ...   
563  UOV-Ip-classic          RSA-3072              1   278832       512   
564  UOV-Ip-classic           Ed25519              1   278464       192   
565   EdDSA-Ed25519          RSA-2048    Pre-Quantum      304       320   
566   EdDSA-Ed25519          RSA-3072    Pre-Quantum      432       448   
567   EdDSA-Ed25519           Ed25519    Pre-Quantum       64       128   

     signing (cycles)  verification (cycles)  signing (ms)  verification (ms)  \
0          1135600

In [5]:
# Format the output for use in table 4.3 and 4.5.

def format_size(size_in_bytes):
    if size_in_bytes < 1_000_000_000:
        return f"{size_in_bytes / 1_000_000:.0f} MB"
    else:
        return f"{size_in_bytes / 1_000_000_000:.1f} GB"


formatted_table = df[
    [
        "Scheme",
        "Parameterset",
        "pk size",
        "sig size",
        "Total size (B)",
        "Download time (high) (s)",
        "Download time (low) (s)",
        "Verification time (s)",
        "Signing time (s)",
    ]
].copy()

formatted_table["Total size"] = formatted_table["Total size (B)"].apply(format_size)
formatted_table["Download time (high) (s)"] = formatted_table["Download time (high) (s)"].round(1)
formatted_table["Download time (low) (s)"] = formatted_table["Download time (low) (s)"].round(1)
formatted_table["Verification time (s)"] = formatted_table["Verification time (s)"].round(1)


In [6]:
# Output for table 4.3.
filtered_schemes = ["RSA", "EdDSA", "ML-DSA", "Falcon", "SLH-DSA", "MAYO", "SNOVA", "HAWK", "SQIsign", "FAEST"]
filtered_formatted_table = formatted_table.loc[formatted_table["Scheme"].isin(filtered_schemes)]
filtered_formatted_table = filtered_formatted_table.iloc[::-1]

with pd.option_context("display.max_rows", None, "display.max_columns", None, "display.max_colwidth", None):
    print(
        filtered_formatted_table[
            [
                "Scheme",
                "Parameterset",
                "pk size",
                "sig size",
                "Total size",
                "Download time (high) (s)",
                "Download time (low) (s)",
                "Verification time (s)",
            ]
        ].to_string(index=False)
    )

 Scheme Parameterset  pk size  sig size Total size  Download time (high) (s)  Download time (low) (s)  Verification time (s)
  EdDSA      Ed25519       32        64     592 MB                     169.6                     10.2                   37.6
SQIsign            I       65       148     671 MB                     192.3                     11.6                 1473.3
SQIsign          III       97       224     744 MB                     213.1                     12.9                 5373.3
SQIsign            V      129       292     810 MB                     232.0                     14.0                10313.3
SLH-DSA   SHAKE-128f       32     17088    14.0 GB                    4015.9                    242.6                 3729.5
SLH-DSA   SHAKE-192f       48     35664    28.7 GB                    8214.6                    496.3                 5742.2
SLH-DSA   SHAKE-256f       64     49856    39.9 GB                   11422.9                    690.1                 5744.9


In [7]:
# Format for use in table 4.4.
print(filtered_formatted_table[["Scheme", "Parameterset", "Signing time (s)"]].to_string(index=False))

 Scheme Parameterset  Signing time (s)
  EdDSA      Ed25519          0.000017
SQIsign            I          0.040640
SQIsign          III          0.123680
SQIsign            V          0.203000
SLH-DSA   SHAKE-128f          0.095918
SLH-DSA   SHAKE-192f          0.154745
SLH-DSA   SHAKE-256f          0.305577
SLH-DSA   SHAKE-128s          1.873028
SLH-DSA   SHAKE-256s          2.834109
SLH-DSA   SHAKE-192s          3.236568
  SNOVA    (37 17 2)          0.000135
  SNOVA     (25 8 3)          0.000148
  SNOVA     (24 5 4)          0.000123
  SNOVA    (56 25 2)          0.000386
  SNOVA    (49 11 3)          0.000546
  SNOVA    (37 8 14)          0.000475
  SNOVA    (75 33 2)          0.000922
  SNOVA    (66 15 3)          0.001419
  SNOVA    (60 10 4)          0.001244
    RSA         2048          0.010800
   MAYO          one          0.000188
   MAYO          two          0.000114
   MAYO        three          0.000407
   MAYO         five          0.000955
   HAWK          512     

In [8]:
# Output for table 4.5.
filtered_schemes = ["Falcon-512", "HAWK-512", "MAYO-one", "SNOVA-(24 5 4)", "ML-DSA-44"]
filtered_formatted_table = formatted_table.loc[formatted_table["Scheme"].isin(filtered_schemes)]
filtered_formatted_table = filtered_formatted_table.iloc[::-1]


with pd.option_context("display.max_rows", None, "display.max_columns", None, "display.max_colwidth", None):
    print(
        filtered_formatted_table[
            [
                "Scheme",
                "Parameterset",
                "pk size",
                "sig size",
                "Total size",
                "Download time (high) (s)",
                "Download time (low) (s)",
                "Verification time (s)",
            ]
        ].to_string(index=False)
    )

        Scheme Parameterset  pk size  sig size Total size  Download time (high) (s)  Download time (low) (s)  Verification time (s)
SNOVA-(24 5 4)      Ed25519     1048       312     1.2 GB                     340.1                     20.5                   84.9
SNOVA-(24 5 4)     RSA-3072     1416       632     1.6 GB                     453.8                     27.4                   75.9
SNOVA-(24 5 4)     RSA-2048     1288       504     1.4 GB                     410.5                     24.8                   60.3
      MAYO-one      Ed25519     1452       518     1.5 GB                     432.1                     26.1                   81.8
      MAYO-one     RSA-3072     1820       838     1.9 GB                     545.9                     33.0                   72.9
      MAYO-one     RSA-2048     1692       710     1.8 GB                     502.6                     30.4                   57.3
      HAWK-512      Ed25519     1056       619     1.4 GB                   